# Imports

In [1]:
# Print every output from a cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# EDA
import numpy as np
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

# Plot pandas objects using plotly.
pd.options.plotting.backend = "plotly"

import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

# Data Source

### Data sourced from The Bureau of Labor Statistics public database ( LABSTAT ): https://download.bls.gov
<br> BLS Average Price Data 2022: https://download.bls.gov/pub/time.series/ap/

# Metadata

### Average Price Description: 

Average consumer prices are calculated for household
fuel, motor fuel, and food items from prices collected for the Consumer
Price Index (CPI). Average prices are best used to measure the price level
in a particular month, not to measure price change over time. It is more
appropriate to use CPI index values for the particular item categories to measure
price change.

### Average Price Database Files: 

    ap.data.0.Current		- All current year-to-date data
	ap.data.1.HouseholdFuels	- All household fuels data
	ap.data.2.Gasoline		- All gasoline data
	ap.data.3.Food			- All food data
	ap.area				- Area codes		mapping file
	ap.contacts			- Contacts for ap survey  
	ap.footnote			- Footnote codes	mapping file
	ap.item				- Item codes		mapping file
	ap.period			- Period codes		mapping file
	ap.series			- All series and their beginning and end Dates
	ap.txt				- General information

### Average Price Database Elements:
    
    Data Element	Length		Value(Example)			Description
    area_code	4		A100			Unique code used to identify
							a specific geographic area.
    area_name	80		Text			Name of specific geographic
				Ex: MIAMI, FLA		area.
    begin_period	3		M02			Identifies first data observation
				Ex: M02=Feb		within the first year for which
				(M=Monthly, M13= 	data is available for a given time
				Annual Avg)		series.					
						
    begin_year	4		1990			Identifies first year for which
 							data is available for a given time
							series.
						
    end_period	3		M06			Identifies last data observation 
 				Ex: M06=June		within the last year for which
				(M=Monthly, M13=	data is available for a given time
				Annual Avg)		series.
						
    end_year	4		2000			Identifies last year for which data
 							is available for a given time
							series.
							
    footnote_code	1		C			Identifies footnote for the data 
							series.

    footnote_codes	10		It varies		Identifies footnotes for the data 
							series.	
							
    footnote_text	100		Text			Contains the text of the footnote.

    item_code	7		712211			Identifies item for which
 							data observations pertain.

    item_name	100		Text			Full names of items.
					
    period_abbr	5		JAN			Abbreviation of period name.
 					
    period		3		M01			Identifies period for which data 
 				Ex: M01=Jan		is observed.
				(M=Monthly, M13=	
				Annual Avg)
 				
    period_name	20		Text			Full name of period to which
 				Ex: January		the data observation refers.
				

    series_id	17		APU0000701111		Code identifying the specific 
 							series.

    value		12		10.124			Average price of item.
	    
    year		4		1990			Identifies year of observation.

# Data Acquisition

### For latest data, pull from the BLS website:

In [2]:
# Define URL strings.
url = 'https://download.bls.gov/pub/time.series/'
ap = 'ap/ap.'

# Define BLS API function.
def blsAPI(string):
    return(pd.read_csv(url+ap+string,sep='\t'))

# Example dataframe for pulling ap.seasonal file from the BLS website:
example_df = blsAPI('seasonal')
example_df.head()

,seasonal_code,seasonal_text
0,S,Seasonally Adjusted
1,U,Not Seasonally Adjusted


## Average Price Data

### For the latest data, pull from LABSTAT the data file ( data.0.Current ) and the significant mapping files ( area, item, period, series; footnote file is empty ):

In [3]:
ap_df = blsAPI('data.0.Current')
area_df = blsAPI('area')
item_df = blsAPI('item')
period_df = blsAPI('period')
series_df = blsAPI('series')

# EDA

## ap_df

In [4]:
ap_df.head()
ap_df.info()

,series_id,year,period,value,footnote_codes
0,APU0000701111,1995,M01,0.238,NaN
1,APU0000701111,1995,M02,0.242,NaN
2,APU0000701111,1995,M03,0.242,NaN
3,APU0000701111,1995,M04,0.236,NaN
4,APU0000701111,1995,M05,0.244,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189044 entries, 0 to 189043
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   series_id          189044 non-null  object 
 1   year               189044 non-null  int64  
 2   period             189044 non-null  object 
 3          value       189044 non-null  object 
 4   footnote_codes     0 non-null       float64
dtypes: float64(1), int64(1), object(3)
memory usage: 7.2+ MB


### The series_id and value feature names contain tabs; let's remove them.

In [5]:
ap_df.columns
ap_df = ap_df.rename(columns={'series_id        ':'series_id', '       value':'value'})
ap_df.columns

Index(['series_id        ', 'year', 'period', '       value',
       'footnote_codes'],
      dtype='object')

Index(['series_id', 'year', 'period', 'value', 'footnote_codes'], dtype='object')

### The footnote_codes values are all null; let's drop this feature.

In [6]:
ap_df.head()
ap_df = ap_df.drop(['footnote_codes'],axis=1)
ap_df.head()

,series_id,year,period,value,footnote_codes
0,APU0000701111,1995,M01,0.238,NaN
1,APU0000701111,1995,M02,0.242,NaN
2,APU0000701111,1995,M03,0.242,NaN
3,APU0000701111,1995,M04,0.236,NaN
4,APU0000701111,1995,M05,0.244,NaN


,series_id,year,period,value
0,APU0000701111,1995,M01,0.238
1,APU0000701111,1995,M02,0.242
2,APU0000701111,1995,M03,0.242
3,APU0000701111,1995,M04,0.236
4,APU0000701111,1995,M05,0.244


### The value feature is cast as a string object for some reason; let's convert it to a float for visualization purposes later. These values also have a leading whitespace, which needs to be removed first.

In [7]:
# Leading whitespace.
ap_df['value'][0]

'       0.238'

In [8]:
# Remove leading whitespace for all examples.
ap_df['value'] = ap_df['value'].str.strip()
ap_df['value'][:5].to_list()

['0.238', '0.242', '0.242', '0.236', '0.244']

In [9]:
# Float conversion wasn't working, so let's search for non-numerical characters.
# Some values are listed as '-'; let's drop those first.
ap_df[ap_df['value']=='-']
ap_df = ap_df[ap_df['value']!='-']
ap_df[ap_df['value']=='-']

,series_id,year,period,value
10623,APU0000710211,2018,M11,-
48494,APU0200710211,2018,M11,-
78341,APU030072511,2018,M11,-
78342,APU030072511,2018,M12,-
83882,APU035072511,2018,M11,-
83883,APU035072511,2018,M12,-
160766,APUS30072511,2018,M11,-
160767,APUS30072511,2018,M12,-


,series_id,year,period,value


In [10]:
# Now we can convert the values to type float.
ap_df.info()
ap_df['value'] = ap_df['value'].apply(float)
ap_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189036 entries, 0 to 189043
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   series_id  189036 non-null  object
 1   year       189036 non-null  int64 
 2   period     189036 non-null  object
 3   value      189036 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.2+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 189036 entries, 0 to 189043
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   series_id  189036 non-null  object 
 1   year       189036 non-null  int64  
 2   period     189036 non-null  object 
 3   value      189036 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.2+ MB


### There are 1048 unique series_id values.

In [11]:
ap_df.nunique()

series_id     1048
year            28
period          12
value        26191
dtype: int64

## area_df

In [12]:
area_df.head()
area_df.info()

,area_code,area_name
0,0000,U.S. city average
1,0100,Northeast
2,0110,New England
3,0120,Middle Atlantic
4,0200,Midwest


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   area_code  74 non-null     object
 1   area_name  74 non-null     object
dtypes: object(2)
memory usage: 1.3+ KB


### All of the area codes are 4 digits long.

In [13]:
area_df['area_code'].apply(len).value_counts()

4    74
Name: area_code, dtype: int64

### There are 74 unique area codes, each corresponding to some region ( the meaning of some, like 'West - Size Class A', are not readily apparent; size class may have to do with population ).

In [14]:
area_df.nunique()

area_code    74
area_name    74
dtype: int64

In [15]:
area_df['area_name'].unique()

array(['U.S. city average', 'Northeast', 'New England', 'Middle Atlantic',
       'Midwest', 'East North Central', 'West North Central', 'South',
       'South Atlantic', 'East South Central', 'West South Central',
       'West', 'Mountain', 'Pacific', 'Pittsburgh, PA',
       'Buffalo-Niagara Falls, NY', 'Scranton, PA', 'Cleveland-Akron, OH',
       'Milwaukee-Racine, WI', 'Cincinnati-Hamilton, OH-KY-IN',
       'Kansas City, MO-KS', 'Washington-Baltimore, DC-MD-VA-WV',
       'Washington, DC-MD-VA', 'Baltimore, MD',
       'Los Angeles-Riverside-Orange County, CA', 'Portland-Salem, OR-WA',
       'City size B', 'Northeast size B', 'North Central size B',
       'South size B', 'West size B', 'City size C', 'Northeast size C',
       'North Central size C', 'South size C', 'West size C',
       'Size Class D', 'Northeast - Size Class D',
       'Midwest - Size Class D', 'South - Size Class D',
       'West - Size Class D', 'Size Class B/C',
       'Northeast - Size Class B/C', 'Midwes

## item_df

In [16]:
item_df.head()
item_df.info()

,item_code,item_name
0,701111,"Flour, white, all purpose, per lb. (453.6 gm)"
1,701311,"Rice, white, long grain, precooked (cost per p..."
2,701312,"Rice, white, long grain, uncooked, per lb. (45..."
3,701321,Spaghetti (cost per pound/453.6 grams)
4,701322,"Spaghetti and macaroni, per lb. (453.6 gm)"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item_code  160 non-null    object
 1   item_name  160 non-null    object
dtypes: object(2)
memory usage: 2.6+ KB


### There are 160 unique item codes, each corresponding to some product.

In [17]:
item_df.nunique()
item_df['item_name'].unique()

item_code    160
item_name    160
dtype: int64

array(['Flour, white, all purpose, per lb. (453.6 gm)',
       'Rice, white, long grain, precooked (cost per pound/453.6 grams)',
       'Rice, white, long grain, uncooked, per lb. (453.6 gm)',
       'Spaghetti (cost per pound/453.6 grams)',
       'Spaghetti and macaroni, per lb. (453.6 gm)',
       'Bread, white, pan, per lb. (453.6 gm)',
       'Bread, French, per lb. (453.6 gm)',
       'Bread, rye, pan (cost per pound/453.6 grams)',
       'Bread, whole wheat, pan, per lb. (453.6 gm)',
       'Bread, wheat blend, pan (cost per pound/453.6 grams)',
       'Rolls, hamburger (cost per pound/453.6 grams)',
       'Cupcakes, chocolate (cost per pound/453.6 grams)',
       'Cookies, chocolate chip, per lb. (453.6 gm)',
       'Crackers, soda, salted, per lb. (453.6 gm)',
       'Ground chuck, 100% beef, per lb. (453.6 gm)',
       'Ground beef, 100% beef, per lb. (453.6 gm)',
       'Ground beef, lean and extra lean, per lb. (453.6 gm)',
       'Chuck roast, USDA Choice, bone-in, per l

## period_df

### There are 13 unique periods, 12 corresponding to the months of the year, and 1 corresponding to an annual average.

In [18]:
period_df

,period,period_abbr,period_name
0,M01,JAN,January
1,M02,FEB,February
2,M03,MAR,March
3,M04,APR,April
4,M05,MAY,May
5,M06,JUN,June
6,M07,JUL,July
7,M08,AUG,August
8,M09,SEP,September
9,M10,OCT,October


## series_df

In [19]:
series_df.head()
series_df.info()

,series_id,area_code,item_code,series_title,footnote_codes,begin_year,begin_period,end_year,end_period
0,APU0000701111,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",NaN,1980,M01,2022,M06
1,APU0000701311,0000,701311,"Rice, white, long grain, precooked (cost per p...",NaN,1980,M01,1981,M12
2,APU0000701312,0000,701312,"Rice, white, long grain, uncooked, per lb. (45...",NaN,1980,M01,2022,M06
3,APU0000701321,0000,701321,Spaghetti (cost per pound/453.6 grams) in U.S....,NaN,1980,M01,1981,M03
4,APU0000701322,0000,701322,"Spaghetti and macaroni, per lb. (453.6 gm) in ...",NaN,1984,M01,2022,M06


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482 entries, 0 to 1481
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   series_id          1482 non-null   object 
 1   area_code          1482 non-null   object 
 2   item_code          1482 non-null   object 
 3   series_title       1482 non-null   object 
 4   footnote_codes     0 non-null      float64
 5   begin_year         1482 non-null   int64  
 6   begin_period       1482 non-null   object 
 7   end_year           1482 non-null   int64  
 8   end_period         1482 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 104.3+ KB


### The series_id feature name contains a tab whitespace; let's remove it.

In [20]:
series_df.columns
series_df = series_df.rename(columns={'series_id        ':'series_id'})
series_df.columns

Index(['series_id        ', 'area_code', 'item_code', 'series_title',
       'footnote_codes', 'begin_year', 'begin_period', 'end_year',
       'end_period'],
      dtype='object')

Index(['series_id', 'area_code', 'item_code', 'series_title', 'footnote_codes',
       'begin_year', 'begin_period', 'end_year', 'end_period'],
      dtype='object')

### There are 1482 unique series_id values, which appear to correspond to a unique series_title value, which lists the item name, the area, and an additional statistical description.

In [21]:
series_df.nunique()
series_df['series_title'].unique()

series_id         1482
area_code           74
item_code          160
series_title      1482
footnote_codes       0
begin_year          30
begin_period        12
end_year            43
end_period          12
dtype: int64

array(['Flour, white, all purpose, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted',
       'Rice, white, long grain, precooked (cost per pound/453.6 grams) in U.S. city average, average price, not seasonally adjusted',
       'Rice, white, long grain, uncooked, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted',
       ...,
       'Gasoline, unleaded midgrade, per gallon/3.785 liters in Urban Alaska, average price, not seasonally adjusted',
       'Gasoline, unleaded premium, per gallon/3.785 liters in Urban Alaska, average price, not seasonally adjusted',
       'Gasoline, all types, per gallon/3.785 liters in Urban Alaska, average price, not seasonally adjusted'],
      dtype=object)

### All of the series_df data is not seasonally adjusted.

In [22]:
# series_id 'APS' prefix designates seasonally adjusted average price value.
len(series_df[series_df['series_id'].str.startswith('APS')])

# series_id 'APU' prefix designates not seasonally adjusted average price value.
len(series_df[series_df['series_id'].str.startswith('APU')])

0

1482

### The series_title values are of the form: '{item_name} in {area_name}, average price, not seasonally adjusted'. 

In [23]:
series_df['series_title'][0]

'Flour, white, all purpose, per lb. (453.6 gm) in U.S. city average, average price, not seasonally adjusted'

In [24]:
series_df[series_df['series_title'].str.contains('flour',case=False,na=False)]

,series_id,area_code,item_code,series_title,footnote_codes,begin_year,begin_period,end_year,end_period
0,APU0000701111,0000,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",NaN,1980,M01,2022,M06
160,APU0100701111,0100,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",NaN,1980,M01,2016,M10
311,APU0200701111,0200,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",NaN,1980,M01,2019,M10
475,APU0300701111,0300,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",NaN,1980,M01,2022,M06
647,APU0400701111,0400,701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",NaN,1980,M01,2022,M05


In [25]:
items = []
for string in item_df['item_name'].unique():
    items.append(string.split(',')[0])
set(items)

{'All Ham (Excluding Canned Ham and Luncheon Slices)',
 'All Other Pork (Excluding Canned Ham and Luncheon Slices)',
 'All Pork Chops',
 'All Uncooked Beef Roasts',
 'All Uncooked Beef Steaks',
 'All Uncooked Other Beef (Excluding Veal)',
 'All soft drinks',
 'All uncooked ground beef',
 'American processed cheese',
 'Apple Sauce',
 'Apples',
 'Automotive diesel fuel',
 'Bacon',
 'Bananas',
 'Beans',
 'Beef for stew',
 'Beef liver (cost per pound/453.6 grams)',
 'Bologna',
 'Bourbon whiskey',
 'Bread',
 'Broccoli',
 'Butter',
 'Cabbage',
 'Carrots',
 'Celery',
 'Cheddar cheese',
 'Cherries',
 'Chicken',
 'Chicken breast',
 'Chicken legs',
 'Chops',
 'Chuck roast',
 'Coffee',
 'Cola',
 'Cookies',
 'Corn',
 'Corn on the cob',
 'Crackers',
 'Cucumbers',
 'Cupcakes',
 'Eggs',
 'Electricity per 500 KWH',
 'Electricity per KWH',
 'Flour',
 'Frankfurters',
 'Fuel oil #2 per gallon (3.785 liters)',
 'Gasoline',
 'Grapefruit',
 'Grapes',
 'Ground beef',
 'Ground chuck',
 'Ham',
 'Hard candy',
 

### Let's check the first and last records in the series_df, and see if they exist in the item_df and area_df ( i.e. the series_title contains the item_name from item_df and area_name from area_df ).

In [26]:
series_df[['series_id','series_title','item_code','area_code']].head(1)
item_df[item_df['item_code']==series_df['item_code'][0]]
area_df[area_df['area_code']==series_df['area_code'][0]]

,series_id,series_title,item_code,area_code
0,APU0000701111,"Flour, white, all purpose, per lb. (453.6 gm) ...",701111,0000


,item_code,item_name
0,701111,"Flour, white, all purpose, per lb. (453.6 gm)"


,area_code,area_name
0,0000,U.S. city average


In [27]:
series_df[['series_id','series_title','item_code','area_code']].tail(1)
item_df[item_df['item_code']==series_df['item_code'][1481]]
area_df[area_df['area_code']==series_df['area_code'][1481]]

,series_id,series_title,item_code,area_code
1481,APUS49G7471A,"Gasoline, all types, per gallon/3.785 liters i...",7471A,S49G


,item_code,item_name
145,7471A,"Gasoline, all types, per gallon/3.785 liters"


,area_code,area_name
73,S49G,Urban Alaska


### Because the number of uniques in the area_df and item_df is the same as the number of uniques in the area_code and item_code columns, we will assume that there is a one-to-one correspondence of these values ( not necessarily true; e.g. the lists [1,2] and [1,3] both contain 2 unique values, but they do not share all unique values ). Are all the codes in area_df and item_df in series_df? We can verify this by the Boolean operations below:

In [28]:
# Are the area code values one-to-one?
set(series_df['area_code'].unique()) == set(area_df['area_code'])

# Are the item code values one-to-one?
set(series_df['item_code'].unique()) == set(item_df['item_code'])

True

True

### There are 434 series_id values in the series_df that are not in the ap_df data. Let's drop these values before further analyses.

In [29]:
# Identify unique series_id values in each DataFrame.
apUniqueIDs = ap_df['series_id'].unique()
seriesUniqueIDs = series_df['series_id'].unique()
len(apUniqueIDs)
len(seriesUniqueIDs)

# Create list of shared series_ids, and drop those not shared from series_df.
sharedIDs = list(set(apUniqueIDs).intersection(seriesUniqueIDs))
series_df = series_df[series_df['series_id'].isin(sharedIDs)]

# Verify that sets of series_id values match.
set(ap_df['series_id'].unique()) == set(series_df['series_id'].unique())

1048

1482

True

### Let's verify the begin_year, begin_period, end_year, and end_period are already contained in the ap_df data.

In [30]:
series_df['begin_year'].min()
ap_df['year'].min()

series_df['end_year'].max()
ap_df['year'].max()

1976

1995

2022

2022

### Unfortunately, it looks like not even the begin_year values match up, so let's just drop these columns from series_df, since there relevance to the actual data is questionable.

### Let's drop all columns besides series_id, area_code, and item_code.

In [31]:
series_df.columns.tolist()

['series_id',
 'area_code',
 'item_code',
 'series_title',
 'footnote_codes',
 'begin_year',
 'begin_period',
 'end_year',
 'end_period']

In [32]:
series_df = series_df.drop(labels=['series_title',
                                   'footnote_codes',
                                   'begin_year',
                                   'begin_period',
                                   'end_year',
                                   'end_period'],axis=1)
series_df.head()

,series_id,area_code,item_code
0,APU0000701111,0000,701111
2,APU0000701312,0000,701312
4,APU0000701322,0000,701322
5,APU0000702111,0000,702111
6,APU0000702112,0000,702112


## df

### Let's create a new DataFrame df that merges the series_df, ap_df, area_df, and item_df features.

### First, let's merge the series_df with the area_code and item_code values from their respective DataFrames.

In [33]:
series_df = pd.merge(series_df,area_df.set_index('area_code')['area_name'],on='area_code')
series_df = pd.merge(series_df,item_df.set_index('item_code')['item_name'],on='item_code')
series_df.head()

,series_id,area_code,item_code,area_name,item_name
0,APU0000701111,0000,701111,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
1,APU0100701111,0100,701111,Northeast,"Flour, white, all purpose, per lb. (453.6 gm)"
2,APU0200701111,0200,701111,Midwest,"Flour, white, all purpose, per lb. (453.6 gm)"
3,APU0300701111,0300,701111,South,"Flour, white, all purpose, per lb. (453.6 gm)"
4,APU0400701111,0400,701111,West,"Flour, white, all purpose, per lb. (453.6 gm)"


### Now we can drop the area_code and item_code features.

In [34]:
series_df.head()
series_df = series_df.drop(labels=['area_code','item_code'],axis=1)
series_df.head()

,series_id,area_code,item_code,area_name,item_name
0,APU0000701111,0000,701111,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
1,APU0100701111,0100,701111,Northeast,"Flour, white, all purpose, per lb. (453.6 gm)"
2,APU0200701111,0200,701111,Midwest,"Flour, white, all purpose, per lb. (453.6 gm)"
3,APU0300701111,0300,701111,South,"Flour, white, all purpose, per lb. (453.6 gm)"
4,APU0400701111,0400,701111,West,"Flour, white, all purpose, per lb. (453.6 gm)"


,series_id,area_name,item_name
0,APU0000701111,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
1,APU0100701111,Northeast,"Flour, white, all purpose, per lb. (453.6 gm)"
2,APU0200701111,Midwest,"Flour, white, all purpose, per lb. (453.6 gm)"
3,APU0300701111,South,"Flour, white, all purpose, per lb. (453.6 gm)"
4,APU0400701111,West,"Flour, white, all purpose, per lb. (453.6 gm)"


### Next, let's merge the average price values into df.

### Let's merge some features into the cpi_df.

### Let's check if there are any average period values.

In [35]:
ap_df['period'].unique()

array(['M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'M08', 'M09',
       'M10', 'M11', 'M12'], dtype=object)

### No average period values, so we don't need to drop them.

### Next, let's merge in the period_name feature from the period_df.

In [36]:
ap_df.head()
ap_df = pd.merge(ap_df,period_df.set_index('period')['period_name'],on='period')
ap_df.head()

,series_id,year,period,value
0,APU0000701111,1995,M01,0.238
1,APU0000701111,1995,M02,0.242
2,APU0000701111,1995,M03,0.242
3,APU0000701111,1995,M04,0.236
4,APU0000701111,1995,M05,0.244


,series_id,year,period,value,period_name
0,APU0000701111,1995,M01,0.238,January
1,APU0000701111,1996,M01,0.262,January
2,APU0000701111,1997,M01,0.309,January
3,APU0000701111,1998,M01,0.299,January
4,APU0000701111,1999,M01,0.297,January


### Next, let's convert the year and period_name into a datetime object feature.

In [37]:
ap_df.head()
ap_df['date'] = pd.to_datetime(ap_df['year'].apply(str) + ap_df['period_name'],format='%Y%B')
ap_df.head()

,series_id,year,period,value,period_name
0,APU0000701111,1995,M01,0.238,January
1,APU0000701111,1996,M01,0.262,January
2,APU0000701111,1997,M01,0.309,January
3,APU0000701111,1998,M01,0.299,January
4,APU0000701111,1999,M01,0.297,January


,series_id,year,period,value,period_name,date
0,APU0000701111,1995,M01,0.238,January,1995-01-01
1,APU0000701111,1996,M01,0.262,January,1996-01-01
2,APU0000701111,1997,M01,0.309,January,1997-01-01
3,APU0000701111,1998,M01,0.299,January,1998-01-01
4,APU0000701111,1999,M01,0.297,January,1999-01-01


### We can drop the year, period, period_abbr, and period_name features now.

In [38]:
ap_df.head()
ap_df = ap_df.drop(['year','period','period_name'],axis=1)
ap_df.head()

,series_id,year,period,value,period_name,date
0,APU0000701111,1995,M01,0.238,January,1995-01-01
1,APU0000701111,1996,M01,0.262,January,1996-01-01
2,APU0000701111,1997,M01,0.309,January,1997-01-01
3,APU0000701111,1998,M01,0.299,January,1998-01-01
4,APU0000701111,1999,M01,0.297,January,1999-01-01


,series_id,value,date
0,APU0000701111,0.238,1995-01-01
1,APU0000701111,0.262,1996-01-01
2,APU0000701111,0.309,1997-01-01
3,APU0000701111,0.299,1998-01-01
4,APU0000701111,0.297,1999-01-01


### Now, let's merge the cpi_df and series_df into one DataFrame df.

In [39]:
df = pd.merge(ap_df,series_df,on='series_id')
df.head()

,series_id,value,date,area_name,item_name
0,APU0000701111,0.238,1995-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
1,APU0000701111,0.262,1996-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
2,APU0000701111,0.309,1997-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
3,APU0000701111,0.299,1998-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
4,APU0000701111,0.297,1999-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"


### We can now drop the series_id feature, and, for visual convenience, we can rename and reorder the columns ( columns ordered from left to right in increasing specificity ).

In [40]:
df.head()
df = df.drop(['series_id'],axis=1)
df = df.rename(columns={'area_name':'area','item_name':'item'})
df = df[['area','date','item','value']]
df.head()

,series_id,value,date,area_name,item_name
0,APU0000701111,0.238,1995-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
1,APU0000701111,0.262,1996-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
2,APU0000701111,0.309,1997-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
3,APU0000701111,0.299,1998-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"
4,APU0000701111,0.297,1999-01-01,U.S. city average,"Flour, white, all purpose, per lb. (453.6 gm)"


,area,date,item,value
0,U.S. city average,1995-01-01,"Flour, white, all purpose, per lb. (453.6 gm)",0.238
1,U.S. city average,1996-01-01,"Flour, white, all purpose, per lb. (453.6 gm)",0.262
2,U.S. city average,1997-01-01,"Flour, white, all purpose, per lb. (453.6 gm)",0.309
3,U.S. city average,1998-01-01,"Flour, white, all purpose, per lb. (453.6 gm)",0.299
4,U.S. city average,1999-01-01,"Flour, white, all purpose, per lb. (453.6 gm)",0.297


# Visualizations

### Let's get an idea of the various items and areas we are working with.

In [41]:
df.info()
df.describe().transpose()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189036 entries, 0 to 189035
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   area    189036 non-null  object        
 1   date    189036 non-null  datetime64[ns]
 2   item    189036 non-null  object        
 3   value   189036 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 7.2+ MB


,count,mean,std,min,25%,50%,75%,max
value,189036.0,10.024339,23.491854,0.0,1.213,2.422,3.829,224.187


In [42]:
df.columns

Index(['area', 'date', 'item', 'value'], dtype='object')

In [43]:
df['area'].nunique()
df['area'].unique().tolist()

65

['U.S. city average',
 'Northeast',
 'New England',
 'Middle Atlantic',
 'Midwest',
 'East North Central',
 'West North Central',
 'South',
 'South Atlantic',
 'East South Central',
 'West South Central',
 'West',
 'Mountain',
 'Pacific',
 'Pittsburgh, PA',
 'Cleveland-Akron, OH',
 'Washington-Baltimore, DC-MD-VA-WV',
 'Washington, DC-MD-VA',
 'Baltimore, MD',
 'Los Angeles-Riverside-Orange County, CA',
 'City size B',
 'Northeast size B',
 'North Central size B',
 'South size B',
 'City size C',
 'Northeast size C',
 'North Central size C',
 'South size C',
 'West size C',
 'Size Class D',
 'Midwest - Size Class D',
 'South - Size Class D',
 'Size Class B/C',
 'Northeast - Size Class B/C',
 'Midwest - Size Class B/C',
 'South - Size Class B/C',
 'West - Size Class B/C',
 'Size Class A',
 'Northeast - Size Class A',
 'Boston-Cambridge-Newton, MA-NH',
 'New York-Newark-Jersey City, NY-NJ-PA',
 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
 'Midwest - Size Class A',
 'Chicago-Naperville

In [44]:
df['item'].nunique()
df['item'].unique().tolist()

116

['Flour, white, all purpose, per lb. (453.6 gm)',
 'Rice, white, long grain, uncooked, per lb. (453.6 gm)',
 'Spaghetti and macaroni, per lb. (453.6 gm)',
 'Bread, white, pan, per lb. (453.6 gm)',
 'Bread, French, per lb. (453.6 gm)',
 'Bread, whole wheat, pan, per lb. (453.6 gm)',
 'Cookies, chocolate chip, per lb. (453.6 gm)',
 'Crackers, soda, salted, per lb. (453.6 gm)',
 'Ground chuck, 100% beef, per lb. (453.6 gm)',
 'Ground beef, 100% beef, per lb. (453.6 gm)',
 'Ground beef, lean and extra lean, per lb. (453.6 gm)',
 'Chuck roast, USDA Choice, bone-in, per lb. (453.6 gm)',
 'Chuck roast, graded and ungraded, excluding USDA Prime and Choice, per lb. (453.6 gm)',
 'Chuck roast, USDA Choice, boneless, per lb. (453.6 gm)',
 'Round roast, USDA Choice, boneless, per lb. (453.6 gm)',
 'Round roast, graded and ungraded, excluding USDA Prime and Choice, per lb. (453.6 gm)',
 'Rib roast, USDA Choice, bone-in, per lb. (453.6 gm)',
 'Steak, T-Bone, USDA Choice, bone-in, per lb. (453.6 gm)'

### Let's create lists for the various types of locations.

In [45]:
cities = ['U.S. city average',
          'Cleveland-Akron, OH',
          'Washington-Baltimore, DC-MD-VA-WV',
          'Washington, DC-MD-VA',
          'Baltimore, MD',
          'Los Angeles-Riverside-Orange County, CA',
          'Boston-Cambridge-Newton, MA-NH',
          'New York-Newark-Jersey City, NY-NJ-PA',
          'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
          'Chicago-Naperville-Elgin, IL-IN-WI',
          'Detroit-Warren-Dearborn, MI',
          'Minneapolis-St.Paul-Bloomington, MN-WI',
          'St. Louis, MO-IL',
          'Washington-Arlington-Alexandria, DC-VA-MD-WV',
          'Miami-Fort Lauderdale-West Palm Beach, FL',
          'Atlanta-Sandy Springs-Roswell, GA',
          'Tampa-St. Petersburg-Clearwater, FL',
          'Baltimore-Columbia-Towson, MD',
          'Dallas-Fort Worth-Arlington, TX',
          'Houston-The Woodlands-Sugar Land, TX',
          'Phoenix-Mesa-Scottsdale, AZ',
          'Denver-Aurora-Lakewood, CO',
          'Los Angeles-Long Beach-Anaheim, CA',
          'San Francisco-Oakland-Hayward, CA',
          'Riverside-San Bernardino-Ontario, CA',
          'Seattle-Tacoma-Bellevue WA',
          'San Diego-Carlsbad, CA',
          'Urban Hawaii',
          'Urban Alaska']

regions = ['Northeast',
           'New England',
           'Middle Atlantic',
           'Midwest',
           'East North Central',
           'West North Central',
           'South',
           'South Atlantic',
           'East South Central',
           'West South Central',
           'West',
           'Mountain',
           'Pacific']

classes = ['City size B',
           'Northeast size B',
           'North Central size B',
           'South size B',
           'City size C',
           'Northeast size C',
           'North Central size C',
           'South size C',
           'West size C',
           'Size Class D',
           'Midwest - Size Class D',
           'South - Size Class D',
           'Size Class B/C',
           'Northeast - Size Class B/C',
           'Midwest - Size Class B/C',
           'South - Size Class B/C',
           'West - Size Class B/C',
           'Size Class A',
           'Northeast - Size Class A',
           'Midwest - Size Class A',
           'South - Size Class A',
           'West - Size Class A']

### Let's generate a DataFrame containing data for the U.S. city average price value of Gasoline, all types.

In [46]:
gas_all_df = df[(df['item'] == 'Gasoline, all types, per gallon/3.785 liters') 
           & (df['area'] == 'U.S. city average')].sort_values(by='date')

# Add monthly and yearly changes in value.
gas_all_df['monthly'] = 100*gas_all_df['value'].pct_change()
gas_all_df['yearly'] = 100*gas_all_df['value'].pct_change(periods=12)

# Drop irrelevant columns.
gas_all_df = gas_all_df.drop(['area','item'],axis=1)
gas_all_df.head()

,date,value,monthly,yearly
22391,1995-01-01,1.190,NaN,NaN
22419,1995-02-01,1.181,-0.756303,NaN
22447,1995-03-01,1.173,-0.677392,NaN
22475,1995-04-01,1.197,2.046036,NaN
22503,1995-05-01,1.256,4.928989,NaN


In [47]:
gas_all_df.iplot(kind='scatter',x='date',y='value',
                 title='U.S. City Average Price, Gasoline, All Types',
                 xTitle='Date',
                 yTitle='Price ($)',
                 color='black')

In [48]:
gas_all_df.iplot(kind='scatter',x='date',y='monthly',
                 title='MoM Change in U.S. City Average Price, Gasoline, All Types',
                 xTitle='Date',
                 yTitle='MoM Change in Price (%)',
                 color='black')

In [49]:
gas_all_df.iplot(kind='scatter',x='date',y='yearly',
                 title='YoY Change in U.S. City Average Price, Gasoline, All Types',
                 xTitle='Date',
                 yTitle='YoY Change in Price (%)',
                 color='black')